***Inventory Planning***

In [ ]:
import warnings
import pandas as pd
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from math import sqrt

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
!pip install pmdarima

# define a function to parse date columns
def parser(x):
    return datetime.strptime(x, '%Y %b')

# load the data
file_name = "/content/monthly sales.xlsx"
data = pd.read_excel(file_name, parse_dates=[['Year', 'Month']], date_parser=parser)

# get unique item names
item_names = data['item_name'].unique()

# loop through each item and generate predictions
results = []
for item_name in item_names:
    # select data for the current item
    item_data = data[data['item_name']==item_name].drop('item_name', axis=1)
    item_data.index = item_data['Year_Month']
    item_data = item_data.drop('Year_Month', axis=1)
    
    # fit ARIMA model
    model = auto_arima(item_data, seasonal=False, suppress_warnings=True)
    model.fit(item_data)
    
    # make predictions for 2023
    predictions = model.predict(n_periods=12, return_conf_int=False)
    dates = pd.date_range(start='2023-01-01', end='2023-12-01', freq='MS')
    
    # create a dataframe with predicted values
    df_pred = pd.DataFrame({'item_name': [item_name]*12, 'Year_Month': dates, 'Sales': predictions})
    df_pred['Year'] = df_pred['Year_Month'].dt.year
    df_pred['Month'] = df_pred['Year_Month'].dt.month
    df_pred = df_pred[['item_name', 'Year', 'Month', 'Sales']]
    results.append(df_pred)
    
# combine all the predictions into a single dataframe
predictions_df = pd.concat(results)

# save the predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)
